In [3]:
import eland as ed
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from elasticsearch import Elasticsearch


In [6]:
ed_flights = ed.read_es('localhost', 'kibana_sample_data_flights')

In [8]:
ed_flights.describe()

,AvgTicketPrice,DistanceKilometers,DistanceMiles,FlightDelayMin,FlightTimeMin,dayOfWeek
count,13059.000000,13059.000000,13059.000000,13059.000000,13059.000000,13059.000000
mean,628.253689,7092.142457,4406.853010,47.335171,511.127842,2.835975
std,266.386661,4578.263193,2844.800855,96.743006,334.741135,1.939365
min,100.020531,0.000000,0.000000,0.000000,0.000000,0.000000
25%,410.008918,2470.545974,1535.126118,0.000000,251.944994,1.000000
50%,640.387285,7612.072403,4729.922470,0.000000,503.148975,3.000000
75%,842.233478,9735.660463,6049.459005,11.080645,720.571142,4.308917
max,1199.729004,19881.482422,12353.780273,360.000000,1902.901978,6.000000


In [10]:
brainwave_df = pd.read_csv('~/Downloads/emotions.csv', index_col=False)

In [12]:
brainwave_df.describe()

,# mean_0_a,mean_1_a,mean_2_a,mean_3_a,mean_4_a,mean_d_0_a,mean_d_1_a,mean_d_2_a,mean_d_3_a,mean_d_4_a,...,fft_740_b,fft_741_b,fft_742_b,fft_743_b,fft_744_b,fft_745_b,fft_746_b,fft_747_b,fft_748_b,fft_749_b
count,2132.000000,2132.000000,2132.000000,2132.000000,2132.000000,2132.000000,2132.000000,2132.000000,2132.000000,2132.000000,...,2132.000000,2132.000000,2132.000000,2132.000000,2132.000000,2132.000000,2132.000000,2132.000000,2132.000000,2132.000000
mean,15.256914,27.012462,-104.975629,13.605898,24.150483,0.025378,0.052282,0.301655,0.036793,0.083567,...,-22.938971,104.946111,-51.973647,-51.973647,104.946111,-6.934144,95.104886,-49.061255,-49.061255,95.104886
std,15.284621,9.265141,206.271960,16.874676,14.187340,17.981796,8.509174,68.098894,17.010031,18.935378,...,298.034311,212.532721,112.160233,112.160233,212.532721,281.040552,203.194976,106.486317,106.486317,203.194976
min,-61.300000,-114.000000,-970.000000,-137.000000,-217.000000,-218.000000,-255.000000,-1360.000000,-203.000000,-553.000000,...,-1180.000000,-921.000000,-504.000000,-504.000000,-921.000000,-1160.000000,-1010.000000,-521.000000,-521.000000,-1010.000000
25%,6.577500,26.075000,-195.000000,4.857500,23.600000,-3.105000,-1.340000,-4.002500,-2.905000,-2.622500,...,-106.500000,-8.365000,-92.900000,-92.900000,-8.365000,-102.500000,-8.837500,-87.150000,-87.150000,-8.837500
50%,14.100000,30.000000,14.950000,15.400000,25.200000,-0.044600,0.132000,0.957500,-0.099750,0.146500,...,83.850000,12.150000,-21.800000,-21.800000,12.150000,89.700000,13.400000,-24.100000,-24.100000,13.400000
75%,27.700000,31.400000,29.600000,26.500000,26.800000,2.920000,1.540000,6.735000,2.535000,2.870000,...,154.000000,177.000000,12.025000,12.025000,177.000000,153.000000,149.250000,10.925000,10.925000,149.250000
max,304.000000,42.300000,661.000000,206.000000,213.000000,402.000000,257.000000,1150.000000,349.000000,444.000000,...,1070.000000,843.000000,1490.000000,1490.000000,843.000000,1180.000000,888.000000,1670.000000,1670.000000,888.000000


In [14]:
label_df = brainwave_df['label']
brainwave_df.drop('label', axis = 1, inplace=True)
brainwave_df.head()

KeyError: 'label'

In [16]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
    from sklearn.model_selection import cross_val_score, train_test_split

pl_random_forest = Pipeline(steps=[('random_forest', RandomForestClassifier())])
scores = cross_val_score(pl_random_forest, brainwave_df, label_df, cv=10,scoring='accuracy')
print('Accuracy for RandomForest : ', scores.mean())

pl_random_forest

Accuracy for RandomForest :  0.9760732025882837


Pipeline(memory=None,
     steps=[('random_forest', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

In [154]:
rfClassifier = pl_random_forest.steps[0][1]
fitted = rfClassifier.fit(brainwave_df, label_df)
rfClassifier.classes_

array(['NEGATIVE', 'NEUTRAL', 'POSITIVE'], dtype=object)

In [239]:
from typing import List
from sklearn.tree import DecisionTreeClassifier
from sklearn.utils.validation import check_is_fitted

def translate_tree_classifier(tree: DecisionTreeClassifier,
                              feature_names: List[str], 
                              classification_labels: List[str]=None) -> dict:
    """ Transform a DecisionTreeClassifier into a format accepted by Elastic ML inference
    
    This simply returns a dict that can then be serialized into JSON for transport
    
    Parameters
    ----------
    
    tree : DecisionTreeClassifier
           The trained/fitted decision tree to transform

    feature_names : array-like
                    The feature names that this decision tree cares about.
                    These names are referenced at inference time to extract 
                    features from the feature map and doc

    classification_labels : array-like, default=None
                            The labels to assign the classification results. 
                            
                            If no labels are supplied, inference returns the numerical value
                            that the tree returns.
                            
    Returns
    -------
    
    dict in the approprate format for Elastic ML inference and storage.
    """
    check_is_fitted(tree,["tree_", "classes_"])
    tree_state = tree.tree_.__getstate__()
    nodes = list()
    for i in range(len(tree_state["nodes"])):
        nodes.append(translate_node(i,tree_state["nodes"][i],tree_state["values"][i]))
    tree_dict = dict()
    tree_dict['tree'] = {
        "target_type": "classification",
        "feature_names": feature_names if feature_names is not None else [], #what to do???
        "tree_structure": nodes,
        "classification_labels": classification_labels if classification_labels is not None else list(tree.classes_),
    }
    return tree_dict 
    
def translate_node(idx,node,value) -> dict:
    if value.shape[0] != 1:
        raise ValueError('unexpected multiple values returned from leaf node ' + idx)
    node_dict = dict()
    node_dict["node_index"] = idx
    # Scikit learn tree nodes are always lte comparisons 
    # See https://github.com/scikit-learn/scikit-learn/blob/b194674c42d54b26137a456c510c5fdba1ba23e0/sklearn/tree/_tree.pyx#L812
    node_dict["decision_type"] = "lte"
    if node[0] == -1: # is leaf node
        if value.shape[1] == 1: # classification requires more than one value, so assume regression 
            node_dict["leaf_value"] = float(value[0][0])
        else: 
            # the classification value, which is the index of the largest value
            node_dict["leaf_value"] = int(np.argmax(value))
    else:
        node_dict["left_child"] = int(node[0])
        node_dict["right_child"] = int(node[1])
        node_dict["split_feature"] = int(node[2])
        node_dict["threshold"] = float(node[3])
    return node_dict
    

In [192]:
from sklearn.ensemble.forest import ForestClassifier

def translate_classifier(classifier,
                         feature_names: List[str], 
                         classification_labels: List[str]=None, 
                         classification_weights: List[float]=None):
    if isinstance(classifier,DecisionTreeClassifier):
        return translate_tree_classifier(classifier,feature_names,classification_labels)
    elif isinstance(classifier,ForestClassifier):
        return translate_ensemble_classifier(classifier,feature_names,classification_labels,classification_weights)
    else:
        raise ValueError('Unexpected classifier model type ' + type(classifier))
        

def translate_ensemble_classifier(ensemble_classifier,
                                  feature_names: List[str], 
                                  classification_labels: List[str]=None, 
                                  classification_weights: List[float]=None):
    """ Transforms an ensemble classifier into the appropriate format for Elast ML utilization
    NOTE: the current ensemble models supported [ForestClassifier]
    Parameters
    ----------
    
    ensemble_classifier : ensemble classifier 
                          The trained/fitted ensemble classifier to transform 

    feature_names : array-like
                    The feature names that this model cares about.
                    These names are referenced at inference time to extract 
                    features from the feature map and doc

    classification_labels : array-like, default=None
                            The labels to assign the classification results. 
                            
                            If no labels are supplied, the classifier is interrogated
                            for the appropriate labels.
    classification_weights : array-like, default=None

                             The weights applied for specific classes on inference
                            
    Returns
    -------
    
    dict in the approprate format for Elastic ML inference and storage.
    
    """
    check_is_fitted(ensemble_classifier,["estimators_", "classes_"])
    models = ensemble_classifier.estimators_
    output_aggregator = dict()
    # sklearn effectively does a weighted vote for ForestClassifiers
    if isinstance(ensemble_classifier,ForestClassifier):
        output_aggregator["weighted_mode"] = { 
            "weights": [1.0] * len(models), 
            "num_classes": len(ensemble_classifier.classes_) }
    else:
        raise ValueError('Unexpected model type ' + type(ensemble_classifier))
    model = dict()
    model['ensemble'] = {
        'target_type': 'classification',
        'feature_names': feature_names,
        'classification_labels': classification_labels if classification_labels is not None else list(ensemble_classifier.classes_),
        'trained_models': [translate_classifier(m,feature_names,classification_labels,classification_weights) for m in models],
        'aggregate_output': output_aggregator,
    }
    if classification_weights is not None:
        model['ensemble']['classification_weights'] = classification_weights
    return model
    
    

In [157]:
translate_classifier(fitted,[])

FileNotFoundError: [Errno 2] File b'iris.data.csv' does not exist: b'iris.data.csv'

In [189]:
iris = pd.read_csv('~/Downloads/iris.data', header = None)
#Renaming the columns
iris.columns = ['sepal_length', 'sepal_width','petal_length','petal_width','species']
print('Shape of the dataset: ' + str(iris.shape))
print(iris.head(100))
print(iris.taill(100))

iris_labels = iris['species']
iris.drop('species', axis = 1, inplace=True)

irisClassifer = RandomForestClassifier()
irisFitted = irisClassifer.fit(iris, iris_labels)
scores = cross_val_score(irisClassifer, iris, iris_labels, cv=10,scoring='accuracy')


Shape of the dataset: (150, 5)
    sepal_length  sepal_width  petal_length  petal_width          species
0            5.1          3.5           1.4          0.2      Iris-setosa
1            4.9          3.0           1.4          0.2      Iris-setosa
2            4.7          3.2           1.3          0.2      Iris-setosa
3            4.6          3.1           1.5          0.2      Iris-setosa
4            5.0          3.6           1.4          0.2      Iris-setosa
..           ...          ...           ...          ...              ...
95           5.7          3.0           4.2          1.2  Iris-versicolor
96           5.7          2.9           4.2          1.3  Iris-versicolor
97           6.2          2.9           4.3          1.3  Iris-versicolor
98           5.1          2.5           3.0          1.1  Iris-versicolor
99           5.7          2.8           4.1          1.3  Iris-versicolor

[100 rows x 5 columns]


AttributeError: 'DataFrame' object has no attribute 'taill'

In [194]:
es_ensemble=translate_classifier(fitted,['sepal_length', 'sepal_width','petal_length','petal_width'])

import json

jsonString = json.dumps({'trained_model':es_ensemble})


import gzip
import base64
compressed_value = base64.b64encode(gzip.compress(bytes(jsonString,'utf-8')))
compressed_value

b'H4sIAJgTM14C/+1b246bMBD9lVWet8jjG3b/oN9QVYgmToLEkghIL6r23wukl90YxRMGlt2WPKwEC/gYe2bOnBl+rOoyzQq3SR4OG5ev3t/9WLmicg+fc9cd1Gm5c3VSfz+2x6t1nlZVts3WaZ0ditX93Wrr0vpUuqRIH1zVXPJxVbljmie5K3b1vr3ifPw125wPj65+9u/z8fnfn+4vh0jy9LPLzw/+UGbVu8rVhyptb+wOv7iyytaH/FD+PZWVu6xo7u8e92x+3XOaSZVurtm1QydVXZ7W7XPPcIoGWpIVG/etOWbNRRu3zqp28r+B5bVrH5a7bZ2s91m+aU5Cc6LMdvu/Z3gL55hndfILdnNOtGPuS1ftD901LDKs+wFYziy3j/d3FwjgKoJ0m3xJ81N7kvn3cix64aEH1gOfX8CXkVEtemFAahvz2IcgsBCkB0GhEMTKH1Ri3xn49yosYO0BjhErDpGy3Y9JLbjSRvoQNAF+jIVvPPi2Bz67gK8iK1v0ptmsWoHoWXFDQG+x9/IeQ8Hbqm+swBGT15dr1wMCKDNA2yv0GKxE7b4+cwFBWDKQ+BlfDSgsat43tH94xD49diMtoeEVhgaN2mi6iwzMKqFjKUDPEBqCrmq6OHHd0PViMreajEEFJxlaccAi4B4CgdhzHBHdOcFqCSYTI14g2WS2z0007DfAe4Hy7DdkrI2I+9yGIhitJhhtTBjXYN+axcX1Ab7Oom2P+RgAtffDLj/A0a5vfkAbL/jWCwJHj9jT3afMzSQtQJYEhR6ibQ984+McsYQqgm4baTBKMB6LHhDo/Aj8BInDcI6q0ePG/rgMx+57hkXnU+AnVGBxCWH3zq3ijclwI/tW3lC2jSVsWM4oN1OSIc4XkjRGXjGEosylOUkUQ7AhhkBgSXYc1YvAksw4PG1kGWsAT5tOxbruOKbTnwIkSYxD1MbXsWAIUwMKzeGUeDUbRwI9Fkek5CiwqAM3Bz6NUgdUt+

In [218]:
from sklearn.ensemble.forest import ForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn import datasets

X, y = datasets.load_boston(return_X_y=True)

# Training classifiers
rfr = RandomForestRegressor(random_state=1, n_estimators=10)
rfr.fit(X, y)

print(rfr.estimators_[0].tree_.__getstate__()['values'][0][0][0])
print(fitted.estimators_[0].tree_.__getstate__()['values'][0])

def translate_tree_regressor(tree: DecisionTreeRegressor,
                             feature_names: List[str]) -> dict:
    """ Transform a DecisionTreeRegressor into a format accepted by Elastic ML inference
    
    This simply returns a dict that can then be serialized into JSON for transport
    
    Parameters
    ----------
    
    tree : DecisionTreeRegressor
           The trained/fitted decision tree to transform

    feature_names : array-like
                    The feature names that this decision tree cares about.
                    These names are referenced at inference time to extract 
                    features from the feature map and doc
    Returns
    -------
    
    dict in the approprate format for Elastic ML inference and storage.
    """
    check_is_fitted(tree,["tree_"])
    tree_state = tree.tree_.__getstate__()
    nodes = list()
    for i in range(len(tree_state["nodes"])):
        nodes.append(translate_node(i,tree_state["nodes"][i],tree_state["values"][i]))
    tree_dict = dict()
    tree_dict['tree'] = {
        "target_type": "regression",
        "feature_names": feature_names if feature_names is not None else [], #what to do???
        "tree_structure": nodes
    }
    return tree_dict 

def translate_regressor(regressor,
                        feature_names: List[str]):
    if isinstance(regressor,DecisionTreeRegressor):
        return translate_tree_regressor(regressor,feature_names)
    elif isinstance(regressor,ForestRegressor):
        return translate_ensemble_regressor(regressor,feature_names)
    else:
        raise ValueError('Unexpected classifier model type ' + type(regressor))
        

def translate_ensemble_regressor(ensemble_regressor,
                                 feature_names: List[str]):
    """ Transforms an ensemble regressor into the appropriate format for Elast ML utilization
    NOTE: the current ensemble models supported [ForestRegressor]
    Parameters
    ----------
    
    ensemble_regressor : ensemble regressor 
                         The trained/fitted ensemble regressor to transform 

    feature_names : array-like
                    The feature names that this model cares about.
                    These names are referenced at inference time to extract 
                    features from the feature map and doc

    Returns
    -------
    
    dict in the approprate format for Elastic ML inference and storage.
    
    """
    check_is_fitted(ensemble_regressor,["estimators_"])
    models = ensemble_regressor.estimators_
    output_aggregator = dict()
    # sklearn effectively does a weighted vote for ForestClassifiers
    if isinstance(ensemble_regressor,ForestRegressor):
        output_aggregator["weighted_sum"] = { 
            # ForestRegressor returns the mean prediction value
            "weights": [1.0/len(models)] * len(models), 
        }
    else:
        raise ValueError('Unexpected model type ' + type(ensemble_regressor))
    model = dict()
    model['ensemble'] = {
        'target_type': 'regression',
        'feature_names': feature_names,
        'trained_models': [translate_regressor(m,feature_names) for m in models],
        'aggregate_output': output_aggregator,
    }
    return model


22.36581027667985
[[45. 56. 49.]]


In [241]:
#np.set_printoptions(precision=10,suppress=True)
print(rfr.predict(X[0:5]))
print(X[0:5])
print(y[0:5])
es_regressor=translate_regressor(rfr,["f1", "f2", "f3", "f4", "f5", "f6", "f7", "f8", "f9", "f10", "f11", "f12", "f13"])

import json
np.set_printoptions()

jsonString = json.dumps({'trained_model':es_regressor})

import gzip
import base64
compressed_value = base64.b64encode(gzip.compress(bytes(jsonString,'utf-8')))
compressed_value

[23.88 21.95 34.46 35.56 36.14]
[[  0.00632  18.00000   2.31000   0.00000   0.53800   6.57500  65.20000
    4.09000   1.00000 296.00000  15.30000 396.90000   4.98000]
 [  0.02731   0.00000   7.07000   0.00000   0.46900   6.42100  78.90000
    4.96710   2.00000 242.00000  17.80000 396.90000   9.14000]
 [  0.02729   0.00000   7.07000   0.00000   0.46900   7.18500  61.10000
    4.96710   2.00000 242.00000  17.80000 392.83000   4.03000]
 [  0.03237   0.00000   2.18000   0.00000   0.45800   6.99800  45.80000
    6.06220   3.00000 222.00000  18.70000 394.63000   2.94000]
 [  0.06905   0.00000   2.18000   0.00000   0.45800   7.14700  54.20000
    6.06220   3.00000 222.00000  18.70000 396.90000   5.33000]]
[24.0 21.6 34.7 33.4 36.2]
{"trained_model": {"ensemble": {"target_type": "regression", "feature_names": ["f1", "f2", "f3", "f4", "f5", "f6", "f7", "f8", "f9", "f10", "f11", "f12", "f13"], "trained_models": [{"tree": {"target_type": "regression", "feature_names": ["f1", "f2", "f3", "f4", "f5

b'...'

In [334]:
import xgboost as xgb
from xgboost import Booster
from sklearn.model_selection import train_test_split


wine = datasets.load_wine() # load data
X_train, X_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size=0.2, random_state=0)
print(wine.feature_names)
dtrain = xgb.DMatrix(X_train, label=y_train)
gbm = xgb.train({'max_depth':6, 'n_estimators':9, 'learning_rate':0.3, 'objective':'multi:softmax', 'num_class': 3},dtrain)
print(gbm.predict(xgb.DMatrix(X_train[0:5])))
print(X_train[0:5])
print(y_train[0:5])
js_tree_list = gbm.get_dump(with_stats=True, dump_format = 'json')
js_trees = [json.loads(s) for s in js_tree_list]
        
def get_feature_id(feature_id: str) -> int:
    if feature_id[0] == "f":
        try:
            return int(feature_id[1:])
        except ValueError:
            raise RuntimeError("Unable to interpret '{0}'".format(feature_id))
    else:
        try:
            return int(feature_id)
        except ValueError:
            raise RuntimeError("Unable to interpret '{0}'".format(feature_id))

def extract_node_id(node_id: str, curr_tree: int) -> int:
    t_id, n_id = node_id.split("-")
    if t_id is None or n_id is None:
        raise RuntimeError(
            "cannot determine node index or tree from '{0}' for tree {1}".format(node_id, curr_tree))
    try:
        t_id = int(t_id)
        n_id = int(n_id)
        if t_id != curr_tree:
            raise RuntimeError("extracted tree id {0} does not match current tree {1}".format(t_id, curr_tree))
        return n_id
    except ValueError:
        raise RuntimeError(
            "cannot determine node index or tree from '{0}' for tree {1}".format(node_id, curr_tree))

def transform_xgboost_trees(model: Booster, feature_names: List[str], base_score: float):
    if model.booster not in {'dart', 'gbtree'}:
        raise ValueError("booster must exist and be of type dart or gbtree")

    tree_table = model.trees_to_dataframe()
    print(model.__dict__)
    transformed_trees = list()
    curr_tree = None
    tree_nodes = list()
    for row in tree_table.itertuples():
        if row.Tree != curr_tree:
            if len(tree_nodes) > 0:
                tree = dict()
                tree["feature_names"] = feature_names
                tree["tree_structure"] = tree_nodes
                transformed_trees.append({'tree': tree})
            curr_tree = row.Tree
            tree_nodes = list()
        node = dict()
        node["node_index"] = row.Node
        if row.Feature == "Leaf":
            node["leaf_value"] = float(row.Gain)
        else:
            feature_idx = get_feature_id(row.Feature)
            node["split_feature"] = feature_idx
            node["left_child"] = extract_node_id(row.Yes, curr_tree)
            node["right_child"] = extract_node_id(row.No, curr_tree)
            node["decision_type"] = "lt" 
            node["threshold"] = float(row.Split)
        tree_nodes.append(node)
    # add last tree
    if len(tree_nodes) > 0:
        tree = dict()
        tree["feature_names"] = feature_names
        tree["tree_structure"] = tree_nodes
        transformed_trees.append({'tree': tree})
    transformed_trees.append({'tree': {'feature_names': feature_names, 'tree_structure':[{'node_index': 0, 'leaf_value': base_score}]}})
    
    return transformed_trees

transformed_trees = transform_xgboost_trees(gbm, ['alcohol', 'malic_acid', 'ash', 'alcalinity_of_ash', 'magnesium', 'total_phenols', 'flavanoids', 'nonflavanoid_phenols', 'proanthocyanins', 'color_intensity', 'hue', 'od280/od315_of_diluted_wines', 'proline'], 0.5)
print(transformed_trees)
print([json.loads(s) for s in gbm.get_dump(with_stats=True, dump_format = 'json')])
print(gbm.trees_to_dataframe())
model = dict()
model['ensemble'] = {
        'target_type': 'regression',
        'feature_names': ['alcohol', 'malic_acid', 'ash', 'alcalinity_of_ash', 'magnesium', 'total_phenols', 'flavanoids', 'nonflavanoid_phenols', 'proanthocyanins', 'color_intensity', 'hue', 'od280/od315_of_diluted_wines', 'proline'],
        'trained_models': transformed_trees,
        'aggregate_output': {"weighted_sum":{}},
    }
jsonString = json.dumps({'trained_model':model})

base64.b64encode(gzip.compress(bytes(jsonString,'utf-8')))

['alcohol', 'malic_acid', 'ash', 'alcalinity_of_ash', 'magnesium', 'total_phenols', 'flavanoids', 'nonflavanoid_phenols', 'proanthocyanins', 'color_intensity', 'hue', 'od280/od315_of_diluted_wines', 'proline']
[2. 1. 1. 2. 0.]
[[ 13.69   3.26   2.54  20.00 107.00   1.83   0.56   0.50   0.80   5.88
    0.96   1.82 680.00]
 [ 12.69   1.53   2.26  20.70  80.00   1.38   1.46   0.58   1.62   3.05
    0.96   2.06 495.00]
 [ 11.62   1.99   2.28  18.00  98.00   3.02   2.26   0.17   1.35   3.25
    1.16   2.96 345.00]
 [ 13.40   3.91   2.48  23.00 102.00   1.80   0.75   0.43   1.41   7.30
    0.70   1.56 750.00]
 [ 13.50   1.81   2.61  20.00  96.00   2.53   2.61   0.28   1.66   3.52
    1.12   3.82 845.00]]
[2 1 1 2 0]
{'feature_names': ['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12'], 'feature_types': None, 'handle': c_void_p(140481475349376), 'booster': 'gbtree', 'best_iteration': 9, 'best_ntree_limit': 10}
[{'tree': {'feature_names': ['alcohol', 'malic_acid',

b'H4sIAB+HNF4C/+2df28bNxKGv0rgv1vd/OKQ069yKATV3tgCZCmQ5faCIt+9w3Xa60Vc3SRdy1uUTgBDm7W5ojgvn3c4ZH69OR032/1wt3483A27mx/e/Xoz7J+Gx592w/jitDneD6f16eOH+vrmONwfh6en7WF/8927m/fD5vR8HNb7zePw5P/875vN7vbwcNjVf3zc7La3683t9q6+2jw9jN92t355vz19XB/erz9ffNzc74en7fNjfXE6nDa79YeHYX/YPY2N7DY/b/aH7d34an/Y/3Hhz3d9OB42+9PD4fbjZr/dj5duD7vDcb3dn/wNeYP10sPzUL8d7qjAvw53jKk+xt1293zyLvjFO+L3X+YPOdz8WJ/nz/0zvknvlOPw0jv/jA4YhvXT6fh8W9/qSwfsvTP8F98N//HX4Dc9fdhtT+vP3eHXkPzibnh/Wt8+bHd39YpfOG7vH/57pd5yN9xu62j6Y4DtTmMfPPgo826st+WUVvDpu3dftIqNVvXLRvmsUYk1SiuFBP5F5y1TpOV01rJGWyYbv9J5yzy2snm//nmze64//z2sCIv5X5bz26XxoPDlg+azBy2xB0Va8cujnrecIi3bWcsIwaZ5hdroH400i+cjESncLlAdFY22c+OzwZQwk3A+v72c3Z5XqixJsg7fQ6NPrdWAcv2TsREf8NUtjD3zPz/iLVBJGYVaLdD57UKaiimXTz9+Gn+iS+X/k0qbTSl5JVpHJ0hbLs8+KwMfoZJjEieziWvJqwlxez1ZhZXJKFalxIQS5xPKFePYNLeF8iyoX0IuYVvgvvgUmXL9IxyQpIvhXBqdQPNpNq0QJmYL+3qlgoszYdeeKKbhbOLjny/WCMs5xmo2m5zwKr2MLLs2q6HHU4zSUGRUZouJj84pPnIB0uLyoM2IsyxARUPaIyRFJLe6rFx6lB7N3XS9uulCHcNEg+FsClYcJq7NErTKJU0ISZMlEjkSCnHMLMmMHo2alN

In [316]:
generated_data = datasets.make_classification(n_features=5)
generated_data

(array([[-1.4249242687, -0.6036625108, -0.1942884055,  1.2590870497,
         -0.5846174147],
        [ 0.9007001619,  0.6235737002, -0.0990978350, -1.5139794975,
         -0.0536022487],
        [-0.6634053016, -0.3842851651,  0.5346164691,  0.8925429148,
         -0.0916679212],
        [ 2.2949942343, -0.3442876870,  0.0130814453,  1.8788709185,
          3.2436374347],
        [ 0.1804841052,  0.1754626661,  0.8962298601, -0.4532580177,
         -0.0990593966],
        [-0.7082154649, -0.3962328317, -1.1007669088,  0.9112592794,
         -0.1223552877],
        [ 1.6224414927, -0.4602292993,  0.7951396739,  1.9717078780,
          2.6722298749],
        [ 1.0395481045, -0.2324874383,  3.1457717833,  1.0781795441,
          1.6030782879],
        [-1.2970991132,  0.1916464905,  1.6901483757, -1.0531872311,
         -1.8281181675],
        [-0.9885879797,  0.2371074342,  0.5400946182, -1.0728544740,
         -1.5524993005],
        [-0.0816170885,  0.9573103893,  0.4792337620, -2.871

In [331]:
dtrain = xgb.DMatrix(generated_data[0], generated_data[1])
gbm = xgb.train({'max_depth':6, 'n_estimators':9, 'learning_rate':0.3, "operator":"binary:logistic"},dtrain)
print(gbm.predict(xgb.DMatrix(generated_data[0][0:5])))
print(generated_data[0][0:5])
print(generated_data[1][0:5])
        
def get_feature_id(feature_id: str) -> int:
    if feature_id[0] == "f":
        try:
            return int(feature_id[1:])
        except ValueError:
            raise RuntimeError("Unable to interpret '{0}'".format(feature_id))
    else:
        try:
            return int(feature_id)
        except ValueError:
            raise RuntimeError("Unable to interpret '{0}'".format(feature_id))

def extract_node_id(node_id: str, curr_tree: int) -> int:
    t_id, n_id = node_id.split("-")
    if t_id is None or n_id is None:
        raise RuntimeError(
            "cannot determine node index or tree from '{0}' for tree {1}".format(node_id, curr_tree))
    try:
        t_id = int(t_id)
        n_id = int(n_id)
        if t_id != curr_tree:
            raise RuntimeError("extracted tree id {0} does not match current tree {1}".format(t_id, curr_tree))
        return n_id
    except ValueError:
        raise RuntimeError(
            "cannot determine node index or tree from '{0}' for tree {1}".format(node_id, curr_tree))

def transform_xgboost_trees(model: Booster, feature_names: List[str], base_score: float):
    if model.booster not in {'dart', 'gbtree'}:
        raise ValueError("booster must exist and be of type dart or gbtree")

    tree_table = model.trees_to_dataframe()
    transformed_trees = list()
    curr_tree = None
    tree_nodes = list()
    for row in tree_table.itertuples():
        if row.Tree != curr_tree:
            if len(tree_nodes) > 0:
                tree = dict()
                tree["feature_names"] = feature_names
                tree["tree_structure"] = tree_nodes
                transformed_trees.append({'tree': tree})
            curr_tree = row.Tree
            tree_nodes = list()
        node = dict()
        node["node_index"] = row.Node
        if row.Feature == "Leaf":
            node["leaf_value"] = float(row.Gain)
        else:
            feature_idx = get_feature_id(row.Feature)
            node["split_feature"] = feature_idx
            node["left_child"] = extract_node_id(row.Yes, curr_tree)
            node["right_child"] = extract_node_id(row.No, curr_tree)
            node["decision_type"] = "lt" 
            node["threshold"] = float(row.Split)
        tree_nodes.append(node)
    # add last tree
    if len(tree_nodes) > 0:
        tree = dict()
        tree["feature_names"] = feature_names
        tree["tree_structure"] = tree_nodes
        transformed_trees.append({'tree': tree})
    transformed_trees.append({'tree': {'feature_names': feature_names, 'tree_structure':[{'node_index': 0, 'leaf_value': 0.5}]}})
    
    return transformed_trees

transformed_trees = transform_xgboost_trees(gbm, ['f0', 'f1', 'f2', 'f3', 'f4'], 0.5)
print(gbm.trees_to_dataframe())
model = dict()
model['ensemble'] = {
        'target_type': 'regression',
        'feature_names': ['f0', 'f1', 'f2', 'f3', 'f4'],
        'trained_models': transformed_trees,
        'aggregate_output': {"weighted_sum":{}},
    }
jsonString = json.dumps({'trained_model':model})

base64.b64encode(gzip.compress(bytes(jsonString,'utf-8')))

[0.0155963 0.9890480 0.0155963 0.9796068 0.9812821]
[[-1.4249242687 -0.6036625108 -0.1942884055  1.2590870497 -0.5846174147]
 [ 0.9007001619  0.6235737002 -0.0990978350 -1.5139794975 -0.0536022487]
 [-0.6634053016 -0.3842851651  0.5346164691  0.8925429148 -0.0916679212]
 [ 2.2949942343 -0.3442876870  0.0130814453  1.8788709185  3.2436374347]
 [ 0.1804841052  0.1754626661  0.8962298601 -0.4532580177 -0.0990593966]]
[0 1 0 1 1]
     Tree  Node    ID Feature     Split   Yes    No Missing       Gain  Cover
0       0     0   0-0      f0 -0.211830   0-1   0-2     0-1  16.615831  100.0
1       0     1   0-1      f1  0.562256   0-3   0-4     0-3   0.787247   44.0
2       0     2   0-2      f0  0.416028   0-5   0-6     0-5   0.925173   56.0
3       0     3   0-3      f4  0.179767   0-7   0-8     0-7   0.829678   43.0
4       0     4   0-4    Leaf       NaN   NaN   NaN     NaN   0.075000    1.0
..    ...   ...   ...     ...       ...   ...   ...     ...        ...    ...
209     9    24  9-24   

b'H4sIACuENF4C/+1bf28bNwz9KoX/7gz+EiXtqwyF4SWXxICbFLYzbCj63UddihXL0WfGsRMHOAdw4avuKFHke4+S7vtst1mu7rvrxdeH6249+/3T91l3v+2+/rnu+h+75ea22y12/3xrv2dX6+V2u7pZXS13q4f72edPs5tuuXvcdIv75ddua03+mN1Afx37b+q/uf+W2Rf7538G+zvMyqZ7MnfE07pusd1tHq/afU9Pu7cnL1b3193f9hus0fbberVb/Hz2z2vr7ma3uLpbra/tAtqFzer27tcVsivX3dVqa8P8b/TrXbO8u9t027uHvtlvMCfEwgDpx+dPz0yjYxqfm+aBaYmZhnlSopSyDC1TZNBpYFmjlgUVKOfkDJod0/LcdB6YLlHTmGvWTIRD009mljeLv5brx37Ic8gJ2oeHrVNkduqgowjRnkLJwJyz01N1bNMgKIdRifGwrJIAKUsZWs/P/WTNUTQhKtVh8/Iit9bI/OMw6jEe9iWj9dVxK8Kwq8hIrbPOwBBD2DDMEwwmCs6zKCkWJ0UxlKM4zBQMp4pK4SzCTgj0EzCIAeon1YUyCfV2mC4UThcujBawyettcuZ1pLPqDG4kZHsnHxwcOSRB4akASy1ldYw76YWS2l/ymlen+ZMnvBAneJkrCF/kaaLRx3/50d8ysfvE7udhd+U+1nSi90P0DpwKMJYYvY/49dL4HarmqjYXNBH8IYKH+oTl+CEYfrS3LsWPhO3lcbxlo0Byw7x6irv2n0JRkh9xhsfyY972af6XgYnnJ54/K88nwZKl1suneTghzePcQlJLxRojechMVEqBIMvv9+rFkXxGLlkrc5Dk6YQk35C9Fq0k9A4sb0GQDGNTzlGSNzc1KFf6ECSfk80qJnd9Rs/P2SMKyk2xPmeSwWaM5UHF/qrlcJDmVfaW5vDC/gxZ/tAN5HUo1coFDFMmln8zlueTsXwbNFvxR5ovnuVFwcDABIlP8wP6SoaKkj